# PageRank

<h3>Autora: Dandara Sousa </h3>

Essa é a resolução da segunda parte do segundo lab da disciplina de Recuperação da Informação. Nessa atividade a ideia é exercitar o algoritmo de PageRank, implementando-o e aplicando num conjunto de dados que representam links entre investidores de bitcoins. Vamos considerar somente as pessoas que receberam notas iguais ou maior a oito. Ou seja, somente as arestas que tenham peso maior ou igual à oito. Após isso, o que se deve fazer é uma visualização dos resultados com a ferramente Gephi. Abaixo, segue as perguntas que devem ser respondidas nesta atividade:
1. Quantas iterações o PageRank precisou rodar até atingir convergência?
2. Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?
3. Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?

In [1]:
import numpy as np
import pandas as pd

Primeiro, vamos organizar os dados. Eles podem ser encontrados no seguinte [link](http://snap.stanford.edu/data/soc-sign-bitcoin-alpha.html).

In [2]:
SOURCE, TARGET, RATING, TIME = 'source', 'target', 'rating', 'time'
df = pd.read_csv("soc-sign-bitcoinalpha.csv", names=[SOURCE, TARGET, RATING, TIME ])

In [3]:
df = df.loc[(df[RATING] >= 8)]
df = df[[SOURCE, TARGET, RATING]]
df.head()

,source,target,rating
0,7188,1,10
1,430,1,10
2,3134,1,10
3,3026,1,10
4,3010,1,10


Nossa próxima meta é construir uma matriz adjacência com os dados. Por ser um grafo precisamos, então, pegar as informações de como os nós se relacionam (para quais nós um nó aponta) e para facilitar a construção da matriz vamos ordenar esse relacionamento.

In [4]:
n_relation = dict()
for source, group_itens in df.groupby([SOURCE]):
    n_relation[source] = len(group_itens)

nodes = sorted(list(set(df.source.append(df.target))))

adjacency_matrix = pd.DataFrame(columns=nodes)

for source in nodes:
    filtered = df.loc[(df.source == source)]
    target_l = filtered.target
    rating_l = filtered.rating
    
    adjacency_matrix_source_row = [0] * len(nodes)
    for t, r in zip(target_l, rating_l):
        index = nodes.index(t)
        adjacency_matrix_source_row[index] = 1 / n_relation[source]
    
    adjacency_matrix.loc[source] = adjacency_matrix_source_row

adjacency_matrix = adjacency_matrix.transpose()

### Page Rank

Implementado a partir do [vídeo](https://www.youtube.com/watch?v=zv4OVNWfVt4).

In [5]:
teleportation_factor = 0.15
B = (teleportation_factor / len(nodes)) * np.matrix(
    [[1] * len(nodes) for i in range(len(nodes))]
    )

M = ((1 - teleportation_factor) * adjacency_matrix.values) + B

page_rank_vector = (1/len(nodes)) * np.matrix(
    [[1] for i in range(len(nodes))]
    )

In [6]:
def pagerank(page_rank_vector, curr_iter):
    
    diff = sum(abs(M * page_rank_vector - page_rank_vector))
    will_converge = diff < 0.001
    
    print('Interation number:', curr_iter)
    print('Diff of page rank vector between iterations:', diff, '\n')
    
    if not will_converge:
        return pagerank(M * page_rank_vector, curr_iter + 1)
    else:
        print('Converged!')
        return M * page_rank_vector

In [7]:
result = pagerank(page_rank_vector, 1)

Interation number: 1
Diff of page rank vector between iterations: [[0.61346461]] 

Interation number: 2
Diff of page rank vector between iterations: [[0.34847521]] 

Interation number: 3
Diff of page rank vector between iterations: [[0.17180354]] 

Interation number: 4
Diff of page rank vector between iterations: [[0.11310951]] 

Interation number: 5
Diff of page rank vector between iterations: [[0.08096262]] 

Interation number: 6
Diff of page rank vector between iterations: [[0.06400668]] 

Interation number: 7
Diff of page rank vector between iterations: [[0.05010319]] 

Interation number: 8
Diff of page rank vector between iterations: [[0.04212086]] 

Interation number: 9
Diff of page rank vector between iterations: [[0.03471744]] 

Interation number: 10
Diff of page rank vector between iterations: [[0.02995771]] 

Interation number: 11
Diff of page rank vector between iterations: [[0.02557733]] 

Interation number: 12
Diff of page rank vector between iterations: [[0.02231991]] 

I

Agora, exportando para csv para utilizarmos no gephi.

In [9]:
node_ids = adjacency_matrix.index.tolist()
index_pr = 0
page_rank_l = list(map(lambda list_pr: list_pr[index_pr], result.tolist()))
nodes_dict = {'Id': node_ids, 'Label': node_ids, 'Modularity Class': page_rank_l}
nodes_df = pd.DataFrame.from_dict(nodes_dict)
nodes_df.to_csv('nodes.csv', index=False)
df.to_csv('edges.csv',  index=False)

![pagerank graph](graph.png "Logo Title Text 1")

### Perguntas

Agora, respondendoa as perguntas:

**Quantas iterações o PageRank precisou rodar até atingir convergência?**

    Foram 99 interações


**Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?**

In [16]:
print('ID', ' -------- ', 'PageRank')

for pr, inv_id in sorted(zip(page_rank_l, node_ids), reverse=True)[:5]:
    print('{:<10}   {:<.8f}'.format(inv_id, pr))

ID  --------  PageRank
2            0.00021287
16           0.00015160
109          0.00014302
461          0.00008390
88           0.00008325


**Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?**

Provavelmente utilizaria para identificar possíveis investidores para transações. Quanto maior o pagerank mais possível seria a transação com o investidor visto que ele pareceria mais propício a trocar as moedas.